#ETL dataset `user_reviews.gz.json`

En este notebook, nuestro objetivo es realizar el proceso de extracción, transformación y carga (ETL) de los datos que contienen información sobre reseñas realizadas por los usuarios de Steama a los videojuegos que consumen, para poder disponer de ellos mediante una API. Este proceso nos permitirá acceder a la información de las reseñas de los usuarios de forma estructurada y actualizada, así como preparar los datos para su posterior análisis y modelado.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y modulos necesarios para nuestro proceso ETL y establecemos configuraciones globales de ser requerido.

In [ ]:
import sys
import pandas as pd
import numpy as np
import gzip
import ast

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas version: 1.5.3
Numpy version: 1.23.5


## 1 Extracción

En esta sección, extraemos los datos del archivo`user_reviews.gz.json` y describimos a detalle su contenido.

### 1.1 Extracción de los datos

En este caso, el conjunto de datos no cumple con un formato válido de JSON, donde el par clave-valor deben estar en comillas dobles.

Por lo tanto, se optó por usar `ast.literal_eval`. Esta función es útil para analizar cadenas JSON que tienen comillas simples en lugar de dobles.

Los datos se extraen descomprimiendo el archivo con el módulo `gzip`, se recorre cada línea del archivo y se interpreta como una estructura de datos de `Python` usando el modulo `ast`. Se almacenan en una lista y se cargan a un Dataframe de `pandas` para observar su contenido.

In [ ]:
# Ruta al dataset
path = '/content/drive/MyDrive/data/raw/user_reviews.json.gz'

# Leemos el archivo usando ast.literal_eval para analizar la cadena JSON
data = []
with gzip.open(path, 'r') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode("utf-8")))

# Convertimos a DataFrame
df_reviews = pd.DataFrame(data)
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


- Hacemos un resumen conciso del Dataframe para observar los tipos de datos por columnas y verificar nulos.

In [ ]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


* Exploramos el contenido de la columna 'review'.

In [ ]:
df_reviews['reviews'][0][:2]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."}]

- Observamos que la estructura de los datos de la columna 'reviews' contiene una lista de diccionarios y procedemos a desanidarlo sin considerar la columna 'user_url' que es irrelevante.

In [ ]:
df_reviews_expanded = pd.json_normalize(data, 'reviews', meta='user_id')
df_reviews_expanded.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637


In [ ]:
df_reviews_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59305 non-null  object
 7   user_id      59305 non-null  object
dtypes: bool(1), object(7)
memory usage: 3.2+ MB


### 1.2 Descripción del los datos.

El conjunto de datos contiene 59305 filas y 9 columnas con información sobre reseñas de usuarios de Steam. A continuación, se describe el contenido de las variables:

- **user_id**: identificador único de usuario.
- **user_url**: URL del perfil del usuario.
- **reviews**: review de usuario en formato Json. Para cada usuario se tienen uno o más diccionarios con el review. Cada diccionario contiene:
    - **funny**: indica si  el review se consideró gracioso.
    - **posted**: fecha de posteo del review en formato Posted April 21, 2011.
    - **last_edited**: fecha de la última edición del review.
    - **item_id**: identificador único del juego.
    - **helpful**: indica si fue útil la información para otros usuarios.
    - **recommend**: booleano que indica si el usuario recomienda o no el juego.
    - **review**: comentarios sobre el juego.

## 2 Transformación

En esta sección, realizamos la limpieza inicial de los datos y las transformaciones necesarias. Esto puede incluir la creación de nuevas columnas a partir de las existentes, la eliminación de duplicados o columnas innecesarias, la gestión de valores nulos o la corrección de tipos de datos.

### 2.1 Gestión de valores nulos

Las columnas ‘funny’ y ‘last_edited’ tienen valores faltantes, pero no figuran como nulos. Posiblemente, están vacías (‘’) por lo que procedemos a comprobarlo.

In [ ]:
df_reviews_expanded[['funny', 'last_edited']].sample(5)

,funny,last_edited
41404,2 people found this review funny,
32862,,
23156,,
21650,,
35517,,


* Como podemos notar, efectivamente están vacías (''). Las reemplazaremos por 'NaN' para verificar la cantidad de valores faltantes que tienen.

In [ ]:
df_reviews_expanded.replace('', np.nan, inplace=True)
df_reviews_expanded.isnull().sum()

funny          51154
posted             0
last_edited    53165
item_id            0
helpful            0
recommend          0
review            30
user_id            0
dtype: int64

* Comprobamos que tienen gran cantidad de nulos. Procedemos a eliminarlas, ya que no son relevantes para nuestros propósitos. También eliminamos la columna 'helpful', que si bien no contiene nulos, también es irrelevante.

In [ ]:
columns = ['funny','last_edited','helpful']
df_reviews_expanded.drop(columns, axis=1, inplace=True)
df_reviews_expanded.columns

Index(['posted', 'item_id', 'recommend', 'review', 'user_id'], dtype='object')

In [ ]:
df_reviews_expanded[df_reviews_expanded['review'].isnull()].head()

,posted,item_id,recommend,review,user_id
3095,Posted March 11.,550,True,NaN,2ZESTY4ME
4616,"Posted September 19, 2014.",550,True,NaN,76561198093337643
15975,"Posted December 30, 2014.",218620,True,NaN,terencemok
20478,Posted March 10.,378041,True,NaN,76561197971285616
22049,"Posted May 23, 2014.",211820,True,NaN,shez13


* Hemos observado que hay usuarios que han recomendado juegos pero no han dejado una reseña. Hemos decidido no eliminarlos porque las recomendaciones pueden ser útiles para nuestro modelo y, en el posterior análisis de sentimientos, simplemente los calificaremos como neutros.

### 2.2 Verificación de duplicados

Comprobamos si tenemos duplicados:

In [ ]:
df_reviews_expanded.duplicated().sum()

874

In [ ]:
df_reviews_expanded[df_reviews_expanded[['item_id','user_id']].duplicated(keep=False)].sort_values(['user_id','item_id'])


,posted,item_id,recommend,review,user_id
13411,"Posted January 5, 2015.",277430,True,this is the best third person game ever that i...,05041129
31985,"Posted January 5, 2015.",277430,True,this is the best third person game ever that i...,05041129
13412,"Posted May 23, 2015.",440,False,this will be the number one game if it have m...,05041129
31986,"Posted May 23, 2015.",440,False,this will be the number one game if it have m...,05041129
13410,"Posted May 18, 2015.",730,True,This game to me it is so good that it is bette...,05041129
...,...,...,...,...,...
43716,"Posted October 31, 2014.",250320,True,"from the creaters of the walking dead, i prese...",yolofaceguy
6752,"Posted September 2, 2014.",261030,True,"this game is awesome,this game is ♥♥♥♥ed up an...",yolofaceguy
43717,"Posted September 2, 2014.",261030,True,"this game is awesome,this game is ♥♥♥♥ed up an...",yolofaceguy
14835,"Posted November 30, 2013.",219640,True,SAVE THE KING!This is one of my favourite LAN ...,zeroblade


Tenemos duplicados que procedemos a eliminar.

In [ ]:
df_reviews_expanded.drop_duplicates(inplace=True)
df_reviews_expanded.shape

(58431, 5)

### 2.3 Extracción del año en la columna `posted`

* Necesitamos extraer el año de la columna 'posted' para las consultas de nuestra API. Exploremos el formato de la columna 'posted':

In [ ]:
df_reviews_expanded['posted']

0         Posted November 5, 2011.
1            Posted July 15, 2011.
2           Posted April 21, 2011.
3            Posted June 24, 2014.
4        Posted September 8, 2013.
                   ...            
59300              Posted July 10.
59301               Posted July 8.
59302               Posted July 3.
59303              Posted July 20.
59304               Posted July 2.
Name: posted, Length: 58431, dtype: object

* Notamos que algunos registros no tienen el año por lo que procederemos de la siguiente manera:
  * Extraemos el año de la columna 'posted' y lo almacenamos en 'posted_year'

In [ ]:
df_reviews_expanded['posted_year'] = df_reviews_expanded['posted'].str.extract('(\d{4})')
df_reviews_expanded[['posted','posted_year']]

,posted,posted_year
0,"Posted November 5, 2011.",2011
1,"Posted July 15, 2011.",2011
2,"Posted April 21, 2011.",2011
3,"Posted June 24, 2014.",2014
4,"Posted September 8, 2013.",2013
...,...,...
59300,Posted July 10.,NaN
59301,Posted July 8.,NaN
59302,Posted July 3.,NaN
59303,Posted July 20.,NaN


* Eliminamos la columna 'posted'

In [ ]:
df_reviews_expanded.drop(columns='posted', inplace=True)
df_reviews_expanded.columns

Index(['item_id', 'recommend', 'review', 'user_id', 'posted_year'], dtype='object')

In [ ]:
df_reviews_expanded['posted_year'].isnull().sum()

9933

* Tenemos datos nulos donde las fechas de la columna 'posted' no tenían año. Los reemplazaremos por 'unknown.'

In [ ]:
df_reviews_expanded['posted_year'] = df_reviews_expanded['posted_year'].fillna('unknown')
df_reviews_expanded['posted_year'].value_counts()

2014       21834
2015       18154
unknown     9933
2013        6713
2012        1201
2011         530
2010          66
Name: posted_year, dtype: int64

In [ ]:
df_reviews_expanded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58431 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   item_id      58431 non-null  object
 1   recommend    58431 non-null  bool  
 2   review       58401 non-null  object
 3   user_id      58431 non-null  object
 4   posted_year  58431 non-null  object
dtypes: bool(1), object(4)
memory usage: 2.3+ MB


## 3 Carga

Finalmente, en esta sección cargamos nuestros datos transformados a un destino interino para su posterior análisis y tratamiento mediante feature engineering. Optamos por almacenarlos en formato parquet con compresion snappy para reducir su tamaño de almacenamiento.

In [ ]:
# Exportamos a parquet
path = 'data/interim/user_reviews.parquet'
df_reviews_expanded.to_parquet(path, engine='pyarrow', compression='snappy')
print(f'El archivo se guardó correctamente en {path}')

El archivo se guardó correctamente en data/interim/user_reviews.parquet


## 4 Referencias

* Steam store. (s/f). Steampowered.com. Recuperado el 25 de octubre de 2023, de https://store.steampowered.com/

* Steam web API. (s/f). Valvesoftware.com. Recuperado el 25 de octubre de 2023, de https://developer.valvesoftware.com/wiki/Steam_Web_API

* Steam community. (s/f). Steamcommunity.com. Recuperado el 25 de octubre de 2023, de https://steamcommunity.com/

